<a href="https://colab.research.google.com/github/galonpy/youtube_llm/blob/main/BERTopic_YouTube_Dislikes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Understanding the YouTube Dislikes Dataset from Kaggle
https://www.kaggle.com/datasets/dmitrynikolaev/youtube-dislikes-dataset

Youtube announced the decision to hide the number of dislikes from users around November 2021. However, the official YouTube Data API allowed you to get information about dislikes until December 13, 2021.

This dataset contains the latest possible information about dislikes, which was collected just before December 13

In [ ]:
  import pandas as pd
  from bertopic import BERTopic

  df = pd.read_csv('/content/youtube_dislike_dataset.csv')

In [ ]:
df.head(2)

,video_id,title,channel_id,channel_title,published_at,view_count,likes,dislikes,comment_count,tags,description,comments
0,--0bCF-iK2E,Jadon Sancho Magical Skills & Goals,UC6UL29enLNe4mqwTfAyeNuw,Bundesliga,2021-07-01 10:00:00,1048888,19515,226,1319,football soccer ftbol alemn Bundesliga season ...,Enjoy the best skills and goals from Jadon San...,"Respect to Dortmund fans,must be sad losing hi..."
1,--14w5SOEUs,Migos - Avalanche (Official Video),UCGIelM2Dj3zza3xyV3pL3WQ,MigosVEVO,2021-06-10 16:00:00,15352638,359277,7479,18729,Migos Avalanche Quality Control Music/Motown R...,"Watch the the official video for Migos - ""Aval...",Migos just makes me want to live my live to th...


In [ ]:
titles = df['title'].tolist()
print(len(titles))

37422


In [ ]:
median_dislikes = df['dislikes'].median()
below_median = df[df['dislikes'] < median_dislikes]['title'].tolist()
above_median = df[df['dislikes'] >= median_dislikes]['title'].tolist()

The above median dataset has some the more disliked videos

In [ ]:
sample_titles = above_median[:10000]


In [ ]:
#from bertopic.representation import KeyBERTInspired

# Fine-tune your topic representations
representation_model = BERTopic()
topic_model = BERTopic(representation_model=representation_model)

In [ ]:
topics, probs = topic_model.fit_transform(sample_titles)

In [ ]:
topic_labels = topic_model.generate_topic_labels(nr_words=3,topic_prefix=False, separator = " - ")
topic_model.set_topic_labels(topic_labels)

In [ ]:
topic_model.get_topic_info()[1:10]

,Topic,Count,Name,CustomName,Representation,Representative_Docs
1,0,378,0_le_des_les_je,le - des - les,"[le, des, les, je, dans, un, et, la, de, avec]",[MICHOU LOUE UNE IMMENSE VILLA DANS LE SUD ! (...
2,1,316,1_de_el_con_que,de - el - con,"[de, el, con, que, en, mi, da, por, los, no]",[NO ELIJAS EL CRISTAL EQUIVOCADO! LOS COMPAS ...
3,2,236,2_minecraft_but_minecrafts_custom,minecraft - but - minecrafts,"[minecraft, but, minecrafts, custom, you, item...","[!? Minecraft, ! MINECRAFT, ..."
4,3,197,3_food_chicken_mcdonalds_recipe,food - chicken - mcdonalds,"[food, chicken, mcdonalds, recipe, theory, bur...","[street food , Food Theory: Can A Slap REALLY ..."
5,4,170,4_oficial_el_video_alejandro,oficial - el - video,"[oficial, el, video, alejandro, rauw, la, los,...","[ANUEL AA, @Ozuna - DIME T (VIDEO OFICIAL), Ra..."
6,5,115,5_nba_lakers_game_full,nba - lakers - game,"[nba, lakers, game, full, at, highlights, nets...",[WARRIORS at LAKERS | NBA PRESEASON FULL GAME ...
7,6,107,6_shorts_english_wait_youtubepartner,shorts - english - wait,"[shorts, english, wait, youtubepartner, hairst...","[ ? ! #shorts, #MATARPANEERPULAO#SHORTS/Gree..."
8,7,107,7_punjabi_song_songs_latest,punjabi - song - songs,"[punjabi, song, songs, latest, 2021, maan, auj...",[Latest Punjabi song 2021 | 1 Hour - Korala Ma...
9,8,101,8_fortnite_season_chapter_event,fortnite - season - chapter,"[fortnite, season, chapter, event, predator, m...","[Fortnite CHAPTER 3 EVENT (best ever), Story T..."


In [ ]:
topic_model.visualize_barchart(top_n_topics=15,custom_labels=True)

In [ ]:
topic_model.visualize_heatmap(top_n_topics=10)

In [ ]:
similar_topics, similarity = topic_model.find_topics("politics", top_n=3); similar_topics

[176, 13, 88]

In [ ]:
#topic_model.get_topic(176)

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP

# Prepare embeddings
#docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(sample_titles, show_progress_bar=False)

# Train BERTopic
topic_model = BERTopic().fit(sample_titles, embeddings)

# Run the visualization with the original embeddings
topic_model.visualize_documents(sample_titles, embeddings=embeddings,topics=list(range(15)),custom_labels=True,height=600)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
